In [1]:
import sys
import subprocess
import importlib
import threading
import socket
import time
import struct
import os
import csv
import json  # [추가] 설정 저장용
import shutil # [추가] 설정 파일 복사용
from datetime import datetime

# ---------------------------------------------------------
# 1. 패키지 자동 설치 함수
# ---------------------------------------------------------
def install_package(module_name, package_name=None):
    if package_name is None:
        package_name = module_name
    try:
        importlib.import_module(module_name)
    except ImportError:
        print(f"Installing {package_name} ...")
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])
            print(f"{package_name} installation completed")
        except subprocess.CalledProcessError as e:
            print(f"{package_name} installation failed (exit code {e.returncode})")
            sys.exit(1)

print("Checking required packages...")
install_package("numpy")
install_package("cv2", "opencv-python")
install_package("PyQt5")

import numpy as np
import cv2
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
from PyQt5.QtCore import *
from PyQt5 import QtWidgets, QtGui

# ---------------------------------------------------------
# 2. 윈도우 소켓 버그 수정용 함수
# ---------------------------------------------------------
def recvall(sock, count):
    buf = b''
    while len(buf) < count:
        try:
            newbuf = sock.recv(count - len(buf))
            if not newbuf: return None
            buf += newbuf
        except socket.timeout:
            raise
        except BlockingIOError:
            continue
    return buf

# ---------------------------------------------------------
# 전역 변수 설정
# ---------------------------------------------------------
HOST_CAM = '192.168.0.60' # [주의] IP 주소 확인 필요
PORT_CAM = 80
PORT_MOT = 81

client_cam = None
client_mot = None

current_command = -1 
running = False 
label_widget = None 
camera_thread = None
motor_thread = None

dirname = ""
f_csv = None
wr = None
labels = ["_0_forward", "_1_right", "_2_left", "_3_stop", "_4_backward"]

# 레터박스(마스킹) 설정 변수
crop_top = 0
crop_bottom = 0
crop_left = 0
crop_right = 0

# 설정 파일 이름
CONFIG_FILE = "mask_config.json"

# ---------------------------------------------------------
# 설정 저장/로드 함수 [추가됨]
# ---------------------------------------------------------
def save_mask_config():
    config = {
        "top": crop_top,
        "bottom": crop_bottom,
        "left": crop_left,
        "right": crop_right
    }
    try:
        with open(CONFIG_FILE, 'w') as f:
            json.dump(config, f)
        print(f"Mask config saved to {CONFIG_FILE}")
    except Exception as e:
        print(f"Failed to save config: {e}")

def load_mask_config():
    global crop_top, crop_bottom, crop_left, crop_right
    if os.path.exists(CONFIG_FILE):
        try:
            with open(CONFIG_FILE, 'r') as f:
                config = json.load(f)
                crop_top = config.get("top", 0)
                crop_bottom = config.get("bottom", 0)
                crop_left = config.get("left", 0)
                crop_right = config.get("right", 0)
            print(f"Loaded mask config: {config}")
            return True
        except Exception as e:
            print(f"Failed to load config: {e}")
    return False

# ---------------------------------------------------------
# 소켓 연결
# ---------------------------------------------------------
def try_connect_esp32(ip_address):
    global client_cam, client_mot
    print(f"Connecting to ESP32 ({ip_address})...")
    
    if client_cam:
        try: client_cam.close()
        except: pass
    if client_mot:
        try: client_mot.close()
        except: pass

    try:
        cam_sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        mot_sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        mot_sock.setsockopt(socket.IPPROTO_TCP, socket.TCP_NODELAY, 1)
        
        cam_sock.settimeout(3)
        mot_sock.settimeout(3)

        cam_sock.connect((ip_address, PORT_CAM))
        mot_sock.connect((ip_address, PORT_MOT))
        
        cam_sock.settimeout(120)
        mot_sock.settimeout(120)
        
        client_cam = cam_sock
        client_mot = mot_sock
        print("Connected successfully!")
        return True
        
    except Exception as e:
        print(f"Connection Failed: {e}")
        return False

# ---------------------------------------------------------
# 폴더 생성
# ---------------------------------------------------------
def createNewFolder():
    global dirname, f_csv, wr
    if f_csv:
        try: f_csv.close()
        except: pass
    
    dirname = datetime.now().strftime("%Y%m%d_%H_%M_%S")

    if not os.path.exists(dirname):
        os.mkdir(dirname)
    
    for label in labels:
        path = os.path.join(dirname, label)
        if not os.path.exists(path):
            os.mkdir(path)

    f_csv = open(os.path.join(dirname, "0_road_labels.csv"),'w', newline='')
    wr = csv.writer(f_csv)
    wr.writerow(["file","label"])
    
    # [추가] 현재 마스킹 설정도 해당 폴더에 백업 저장
    save_mask_config() # 먼저 현재 폴더에 저장
    try:
        shutil.copy(CONFIG_FILE, os.path.join(dirname, CONFIG_FILE))
        print(f"Mask config backed up to {dirname}/{CONFIG_FILE}")
    except Exception as e:
        print(f"Backup failed: {e}")

    print(f"--> New data folder created: {dirname}")

# ---------------------------------------------------------
# 카메라 쓰레드 (레터박스 적용)
# ---------------------------------------------------------
def camMain():
    global current_command, running, dirname, f_csv, wr
    global crop_top, crop_bottom, crop_left, crop_right
    
    t_prev = time.time()
    cnt_frame = 0
    cnt_frame_total = 0
    
    DISPLAY_WIDTH = 640
    DISPLAY_HEIGHT = 480
    
    cmd_prefixes = ['f', 'r', 'l', 's', 'b'] 
    
    while label_widget is None and running:
        time.sleep(0.1)
        
    try:
        if label_widget: label_widget.resize(DISPLAY_WIDTH, DISPLAY_HEIGHT)
    except: pass

    while running:
        if client_cam is None:
            time.sleep(1)
            continue

        try:
            cmd = 12
            cmd = struct.pack('B', cmd)
            client_cam.sendall(cmd) 

            data_len_bytes = recvall(client_cam, 4)
            if not data_len_bytes: continue
            data_len = struct.unpack('I', data_len_bytes)[0]
            
            data = recvall(client_cam, data_len)
            if not data: continue

            np_data = np.frombuffer(data, dtype='uint8')
            frame = cv2.imdecode(np_data, 1)
            if frame is None: continue
            
            frame = cv2.rotate(frame, cv2.ROTATE_180) 
            frame_resized = cv2.resize(frame, (DISPLAY_WIDTH, DISPLAY_HEIGHT), interpolation=cv2.INTER_LINEAR)
            
            # 레터박스(마스킹) 적용
            h, w, _ = frame_resized.shape
            
            if crop_top > 0: frame_resized[:crop_top, :] = 0
            if crop_bottom > 0: frame_resized[h-crop_bottom:, :] = 0
            if crop_left > 0: frame_resized[:, :crop_left] = 0
            if crop_right > 0: frame_resized[:, w-crop_right:] = 0

            qImg = QtGui.QImage(frame_resized.data, w, h, w*3, QtGui.QImage.Format_RGB888)
            pixmap = QtGui.QPixmap.fromImage(qImg.rgbSwapped())
            
            if label_widget is not None:
                label_widget.setPixmap(pixmap)

            # 저장 로직
            if current_command != -1 and f_csv is not None and not f_csv.closed:
                prefix = cmd_prefixes[current_command]
                time_str = datetime.now().strftime("%Y%m%d_%H_%M_%S_%f")
                road_file = f"{prefix}{time_str}.png"
                
                save_path = os.path.join(dirname, labels[current_command])
                cv2.imwrite(os.path.join(save_path, road_file), frame_resized)
                
                if wr:
                    wr.writerow([os.path.join(labels[current_command], road_file), current_command])
                    f_csv.flush()
                cnt_frame_total += 1

            cnt_frame += 1
            t_now = time.time()
            if t_now - t_prev >= 1.0:
                print(f"FPS: {cnt_frame}, Saved: {cnt_frame_total}, CMD: {current_command}, Mask: T{crop_top}/B{crop_bottom}/L{crop_left}/R{crop_right}")
                cnt_frame = 0
                t_prev = t_now
                
        except socket.timeout:
            print("Socket timed out.")
            time.sleep(0.5) 
            continue
        except Exception as e:
            print(f"Cam Error: {e}")
            break

# ---------------------------------------------------------
# 모터 제어
# ---------------------------------------------------------
def send_motor_command(cmd_idx):
    global current_command
    current_command = cmd_idx

def motorMain():
    global current_command, running, client_mot
    while running:
        if client_mot:
            try:
                cmd_char = b'S'
                if current_command == 0: cmd_char = b'F'
                elif current_command == 1: cmd_char = b'R'
                elif current_command == 2: cmd_char = b'L'
                elif current_command == 3: cmd_char = b'S'
                elif current_command == 4: cmd_char = b'B'
                client_mot.sendall(cmd_char)
            except: pass
        time.sleep(0.1)

# ---------------------------------------------------------
# 메인 윈도우
# ---------------------------------------------------------
class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle('RC Car Data Collector (Auto-Save Mask)')
        self.setGeometry(100, 100, 450, 750)
        self.setFocusPolicy(Qt.StrongFocus)

        # [중요] 시작 시 이전 설정 불러오기
        load_mask_config()

        cw = QWidget()
        self.setCentralWidget(cw)
        layout = QVBoxLayout()
        cw.setLayout(layout)

        # 1. 화면
        global label_widget
        label_widget = QLabel("Waiting for Video...")
        label_widget.setAlignment(Qt.AlignCenter)
        label_widget.setStyleSheet("background-color: black; color: white;")
        label_widget.setScaledContents(True)
        label_widget.setMinimumHeight(320)
        layout.addWidget(label_widget)

        # 2. 레터박스 조절 슬라이더
        gb_mask = QGroupBox("Camera Masking (Auto-Saved)")
        gb_layout = QGridLayout()
        gb_mask.setLayout(gb_layout)
        
        self.sliders = {}

        def create_slider(label_text, row, col, max_val, callback, init_val):
            lbl = QLabel(label_text)
            slider = QSlider(Qt.Horizontal)
            slider.setRange(0, max_val)
            slider.setValue(init_val) # 불러온 값으로 초기화
            val_label = QLabel(str(init_val))
            
            def on_change(val):
                val_label.setText(str(val))
                callback(val)
                self.setFocus()
                
            slider.valueChanged.connect(on_change)
            slider.setFocusPolicy(Qt.NoFocus)
            
            gb_layout.addWidget(lbl, row, col)
            gb_layout.addWidget(slider, row, col+1)
            gb_layout.addWidget(val_label, row, col+2)
            return slider
            
        # 불러온 값(init_val)을 적용하여 슬라이더 생성
        self.sliders['top'] = create_slider("Top:", 0, 0, 240, self.set_crop_top, crop_top)
        self.sliders['bottom'] = create_slider("Bottom:", 1, 0, 240, self.set_crop_bottom, crop_bottom)
        self.sliders['left'] = create_slider("Left:", 0, 3, 320, self.set_crop_left, crop_left)
        self.sliders['right'] = create_slider("Right:", 1, 3, 320, self.set_crop_right, crop_right)
        
        layout.addWidget(gb_mask)

        # 3. 버튼 그리드
        btn_layout = QGridLayout()
        layout.addLayout(btn_layout)

        self.btn_fwd = QPushButton("Forward (W)")
        self.btn_left = QPushButton("Left (A)")
        self.btn_right = QPushButton("Right (D)")
        self.btn_stop = QPushButton("Stop (Space)") 
        self.btn_back = QPushButton("Backward (S)") 
        
        for btn in [self.btn_fwd, self.btn_left, self.btn_right, self.btn_stop, self.btn_back]:
            btn.setFixedHeight(50)
            btn.setStyleSheet("font-weight: bold; font-size: 14px;")
            btn.setFocusPolicy(Qt.NoFocus) 

        btn_layout.addWidget(self.btn_fwd, 0, 1)   
        btn_layout.addWidget(self.btn_left, 1, 0)  
        btn_layout.addWidget(self.btn_stop, 1, 1)  
        btn_layout.addWidget(self.btn_right, 1, 2) 
        btn_layout.addWidget(self.btn_back, 2, 1)  

        # 4. 기능 버튼
        self.btn_new_folder = QPushButton("Make New Folder (N)")
        self.btn_new_folder.setFixedHeight(40)
        self.btn_new_folder.setStyleSheet("background-color: #DDDDDD; font-weight: bold;")
        self.btn_new_folder.clicked.connect(self.on_new_folder_clicked)
        self.btn_new_folder.setFocusPolicy(Qt.NoFocus)
        layout.addWidget(self.btn_new_folder)
        
        self.btn_change_ip = QPushButton(f"Change IP Address (Current: {HOST_CAM})")
        self.btn_change_ip.setFixedHeight(40)
        self.btn_change_ip.clicked.connect(self.on_change_ip_clicked)
        self.btn_change_ip.setFocusPolicy(Qt.NoFocus)
        layout.addWidget(self.btn_change_ip)

        # 5. 이벤트 연결
        self.btn_fwd.pressed.connect(lambda: send_motor_command(0))   
        self.btn_left.pressed.connect(lambda: send_motor_command(2))  
        self.btn_right.pressed.connect(lambda: send_motor_command(1)) 
        self.btn_stop.pressed.connect(lambda: send_motor_command(3))  
        self.btn_back.pressed.connect(lambda: send_motor_command(4))  

        self.btn_fwd.released.connect(lambda: send_motor_command(-1))
        self.btn_left.released.connect(lambda: send_motor_command(-1))
        self.btn_right.released.connect(lambda: send_motor_command(-1))
        self.btn_stop.released.connect(lambda: send_motor_command(-1))
        self.btn_back.released.connect(lambda: send_motor_command(-1))

        layout.addWidget(QLabel("Controls: WASD/Arrows to Move, Space to Stop"))
        self.setFocus()

    def set_crop_top(self, val): global crop_top; crop_top = val
    def set_crop_bottom(self, val): global crop_bottom; crop_bottom = val
    def set_crop_left(self, val): global crop_left; crop_left = val
    def set_crop_right(self, val): global crop_right; crop_right = val

    def focusNextPrevChild(self, next): return False

    def on_new_folder_clicked(self):
        createNewFolder()
        QMessageBox.information(self, "New Folder", f"New folder created:\n{dirname}\n(Mask config backed up)")
        self.setFocus()

    def on_change_ip_clicked(self):
        global HOST_CAM, running, camera_thread, motor_thread
        text, ok = QInputDialog.getText(self, 'Change IP', 'Enter ESP32 IP Address:', text=HOST_CAM)
        if ok and text:
            HOST_CAM = text
            self.btn_change_ip.setText(f"Change IP Address (Current: {HOST_CAM})")
            running = False
            if camera_thread: camera_thread.join(timeout=1.0)
            if try_connect_esp32(HOST_CAM):
                QMessageBox.information(self, "Success", "Connected!")
                running = True
                camera_thread = threading.Thread(target=camMain)
                camera_thread.setDaemon(True); camera_thread.start()
                motor_thread = threading.Thread(target=motorMain)
                motor_thread.setDaemon(True); motor_thread.start()
            else:
                QMessageBox.warning(self, "Failed", "Check IP.")
            self.setFocus()

    def keyPressEvent(self, event):
        key = event.key()
        if key == Qt.Key_Escape: self.close(); return
        if key == Qt.Key_N: self.on_new_folder_clicked(); return

        if not event.isAutoRepeat(): 
            if key in [Qt.Key_W, Qt.Key_Up]: self.btn_fwd.setDown(True); send_motor_command(0)
            elif key in [Qt.Key_A, Qt.Key_Left]: self.btn_left.setDown(True); send_motor_command(2)
            elif key in [Qt.Key_D, Qt.Key_Right]: self.btn_right.setDown(True); send_motor_command(1)
            elif key in [Qt.Key_S, Qt.Key_Down]: self.btn_back.setDown(True); send_motor_command(4)
            elif key == Qt.Key_Space: self.btn_stop.setDown(True); send_motor_command(3)

    def keyReleaseEvent(self, event):
        key = event.key()
        if not event.isAutoRepeat():
            if key in [Qt.Key_W, Qt.Key_Up, Qt.Key_A, Qt.Key_Left, Qt.Key_D, Qt.Key_Right, Qt.Key_S, Qt.Key_Down, Qt.Key_Space]:
                self.btn_fwd.setDown(False); self.btn_left.setDown(False)
                self.btn_right.setDown(False); self.btn_stop.setDown(False)
                self.btn_back.setDown(False); send_motor_command(-1)
        event.accept()

    def closeEvent(self, event):
        print("Window closing...")
        # [중요] 종료 시 설정 저장
        save_mask_config()
        event.accept()

# ---------------------------------------------------------
# 실행부
# ---------------------------------------------------------
app = QApplication.instance()
if app is None: app = QApplication(sys.argv)

connected = False
while not connected:
    connected = try_connect_esp32(HOST_CAM)
    if not connected:
        text, ok = QInputDialog.getText(None, 'Connection Failed', 'Enter ESP32 IP Address:', text=HOST_CAM)
        if ok and text: HOST_CAM = text
        else: sys.exit()

if connected:
    createNewFolder()
    mw = MainWindow()
    mw.show()
    running = True
    camera_thread = threading.Thread(target=camMain)
    camera_thread.setDaemon(True); camera_thread.start()
    motor_thread = threading.Thread(target=motorMain)
    motor_thread.setDaemon(True); motor_thread.start()
    try: app.exec_()
    except SystemExit: pass
    finally:
        running = False
        try: client_cam.close() 
        except: pass
        try: client_mot.close() 
        except: pass
        if f_csv:
            try: f_csv.close()
            except: pass
        print("Done.")

Checking required packages...
Connecting to ESP32 (192.168.0.60)...
Connected successfully!
Mask config saved to mask_config.json
Mask config backed up to 20251208_19_40_35/mask_config.json
--> New data folder created: 20251208_19_40_35
Loaded mask config: {'top': 0, 'bottom': 0, 'left': 0, 'right': 0}


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_34804\3799496416.py:495: DeprecationWarning: setDaemon() is deprecated, set the daemon attribute instead
  camera_thread.setDaemon(True); camera_thread.start()
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_34804\3799496416.py:497: DeprecationWarning: setDaemon() is deprecated, set the daemon attribute instead
  motor_thread.setDaemon(True); motor_thread.start()


FPS: 25, Saved: 0, CMD: -1, Mask: T0/B0/L0/R0
FPS: 28, Saved: 0, CMD: -1, Mask: T0/B0/L0/R0
FPS: 28, Saved: 0, CMD: -1, Mask: T0/B0/L0/R0
FPS: 28, Saved: 0, CMD: -1, Mask: T0/B0/L0/R0
FPS: 26, Saved: 0, CMD: -1, Mask: T0/B0/L0/R0
FPS: 28, Saved: 0, CMD: -1, Mask: T0/B0/L0/R0
FPS: 28, Saved: 0, CMD: -1, Mask: T0/B0/L0/R0
FPS: 28, Saved: 0, CMD: -1, Mask: T0/B0/L0/R0
FPS: 29, Saved: 0, CMD: -1, Mask: T0/B0/L0/R0
FPS: 28, Saved: 0, CMD: -1, Mask: T0/B0/L0/R0
FPS: 28, Saved: 0, CMD: -1, Mask: T0/B0/L0/R0
FPS: 28, Saved: 0, CMD: -1, Mask: T0/B0/L0/R0
FPS: 28, Saved: 0, CMD: -1, Mask: T0/B0/L0/R0
FPS: 27, Saved: 4, CMD: -1, Mask: T0/B0/L0/R0
FPS: 28, Saved: 18, CMD: -1, Mask: T0/B0/L0/R0
FPS: 27, Saved: 27, CMD: -1, Mask: T0/B0/L0/R0
FPS: 28, Saved: 27, CMD: -1, Mask: T0/B0/L0/R0
FPS: 28, Saved: 27, CMD: -1, Mask: T0/B0/L0/R0
FPS: 25, Saved: 27, CMD: -1, Mask: T0/B0/L0/R0
FPS: 15, Saved: 29, CMD: -1, Mask: T0/B0/L0/R0
FPS: 9, Saved: 31, CMD: -1, Mask: T0/B0/L0/R0
FPS: 16, Saved: 31, CMD: -1,